In [16]:
# add parent directory to path
import sys
dir_parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(dir_parent)
import os
import json
import winsound
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from tqdm import tqdm
from src.db import DataBase
from src.utils_date import add_days
from src.utils_stocks import get_ls_sym
from src.utils_stocks import get_df_prices
from src.utils_stocks import suppress_stdout
dir_db = os.path.join(dir_parent, 'data', 'db')
dir_db_demo = os.path.join(dir_parent, 'data', 'demo',)

def beeps(n=3):
    for _ in range(n):
        winsound.Beep(200,500)

def db_remove_dups_prices_m(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_m
         WHERE DATE(datetime)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_m
                WHERE DATE(datetime)>='{}'
                GROUP BY sym, datetime)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_prices_d(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_d
         WHERE DATE(date)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_d
                WHERE DATE(date)>='{}'
                GROUP BY sym, date)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_stocks(db):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM stocks
         WHERE ROWID NOT IN
               (SELECT MAX(ROWID)
                  FROM stocks
                 GROUP BY sym)
    '''
    db.execute(q)

ls_init_str = [
    #prices_m
    '''CREATE TABLE IF NOT EXISTS prices_m(
        sym TEXT
        ,datetime TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER
        ,is_reg_hours INTEGER)''',
    
    '''CREATE INDEX IF NOT EXISTS index_prices_m_all
        ON prices_m(sym, date(datetime), is_reg_hours)''',

    '''CREATE INDEX IF NOT EXISTS index_prices_m_date
        ON prices_m(date(datetime))''',

    #prices_d
    '''CREATE TABLE IF NOT EXISTS prices_d(
        sym TEXT
        ,date TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER)''',
   
    '''CREATE INDEX IF NOT EXISTS index_prices_d_date
        ON prices_d(sym, date(date))''',
    
    #stocks
    '''CREATE TABLE IF NOT EXISTS stocks(
        sym TEXT
        ,long_name TEXT
        ,sec TEXT
        ,ind TEXT
        ,quote_type TEXT
        ,fund_family TEXT
        ,summary TEXT
        ,timestamp TEXT)''',

    '''CREATE INDEX IF NOT EXISTS index_stocks
        ON stocks(sym, quote_type)''',
    
    #stocks_error
    '''CREATE TABLE IF NOT EXISTS stocks_error(
        sym TEXT)''',
    
    #proba
    '''CREATE TABLE IF NOT EXISTS proba(
        sym TEXT
        ,datetime TEXT
        ,my_index INTEGER
        ,proba REAL
        ,datetime_update TEXT)''',
]

db = DataBase(ls_init_str, dir_db)
db_demo = DataBase(ls_init_str, dir_db_demo)

# Manual db view/modify

In [17]:
#execute
assert 0
q = '''
ALTER TABLE trading_days
RENAME COLUMN Date TO date
'''
q = '''
drop table trading_days
'''
q='''
ALTER TABLE prices_interday
  RENAME TO prices_d
'''
q='''
drop index index_prices_interday
'''

q='''
    UPDATE prices_m
       SET is_reg_hours = CASE
                           WHEN time(datetime) < time('09:30:00') THEN 0
                           WHEN time(datetime) > time('15:59:00') THEN 0
                           ELSE 1
                           END
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
q='''
    --UPDATE prices_m
       SET datetime = DATETIME(datetime, '-1 hours')
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
db.execute(q)
beeps()

AssertionError: 

In [18]:
#read
q = '''
SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

'''
q='''
    select type, name, tbl_name, sql
    FROM sqlite_master
    WHERE type='index'
'''
q='''
select *
from stocks
limit 10
'''
df = pd.read_sql(q, db.conn)
df

,sym,long_name,sec,ind,quote_type,fund_family,summary,timestamp
0,SPXN,ProShares S&P 500 ex-Financials ETF,None,None,ETF,ProShares,None,2020-06-27 22:20:59.113357
1,BCD,Aberdeen Standard Bloomberg All Commodity Long...,None,None,ETF,Aberdeen Standard Investments,None,2020-06-27 22:21:05.380440
2,JPNL,Direxion Daily Japan Bull 3X Shares,None,None,ETF,Direxion Funds,None,2020-06-27 22:21:08.195438
3,RESE,WisdomTree Emerging Markets ESG Fund,None,None,ETF,WisdomTree,None,2020-06-27 22:21:18.082804
4,PGF,Invesco Financial Preferred ETF,None,None,ETF,Invesco,None,2020-06-27 22:21:20.017096
5,SMLV,SPDR SSGA US Small Cap Low Volatility Index ETF,None,None,ETF,SPDR State Street Global Advisors,None,2020-06-27 22:21:21.999377
6,ERSX,ERShares Non-US Small Cap Fund,None,None,ETF,EntrepreneurShares,None,2020-06-27 22:21:25.487411
7,BNDX,Vanguard Total International Bond Index Fund E...,None,None,ETF,Vanguard,None,2020-06-27 22:21:26.643849
8,SSUS,Day Hagan/Ned Davis Research Smart Sector ETF,None,None,ETF,Day Hagan,None,2020-06-27 22:21:29.839304
9,RDIV,Invesco S&P Ultra Dividend Revenue ETF,None,None,ETF,Invesco,None,2020-06-27 22:21:30.923766


In [5]:
get_df_prices('BYND', '2020-11-28', '2020-12-03')

,sym,datetime,open,high,low,adj_close,volume,is_reg_hours
133,BYND,2020-11-30 05:21:00,141.900000,141.900000,141.900000,141.900000,0,0
134,BYND,2020-11-30 07:07:00,140.360000,140.360000,140.350000,140.350000,0,0
135,BYND,2020-11-30 08:17:00,140.530000,140.530000,140.530000,140.530000,0,0
136,BYND,2020-11-30 08:18:00,140.900000,140.900000,140.900000,140.900000,0,0
137,BYND,2020-11-30 08:33:00,141.180000,141.250000,141.180000,141.250000,0,0
...,...,...,...,...,...,...,...,...
1066,BYND,2020-12-02 10:50:00,135.070007,135.388702,134.800003,135.070007,11643,1
1067,BYND,2020-12-02 10:51:00,134.860001,135.559998,134.800003,135.490005,14073,1
1068,BYND,2020-12-02 10:52:00,135.489899,135.930206,135.445007,135.930206,10986,1
1069,BYND,2020-12-02 10:53:00,135.759995,136.339996,135.759995,136.294998,14608,1


In [19]:
dt_info = yf.Ticker('CDNA').info